<h1><b>A. Nhập</b></h1>

<h3><b>1. Tải các thư viên</b></h3>

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt 

<h3><b>2. Tải data

In [2]:
post_data = pd.read_csv('./dataset/csv_data/post_data.csv')
content_data = pd.read_csv('./dataset/csv_data/content_data.csv')
title_data = pd.read_csv('./dataset/csv_data/title_data.csv')

cleaned_data = pd.DataFrame() # Lưu kết quả cuối vào đây

cleaned_data['post_code'] = post_data['post_code']
cleaned_data['post_date'] = post_data['post_date']

cleaned_data["content"] = post_data["content"]
cleaned_data["title"] = post_data["title"]

cleaned_data["c_rent"] = content_data["rent"]
cleaned_data["t_rent"] = title_data["rent"]

cleaned_data["c_hurried"] = content_data["hurried"]
cleaned_data["t_hurried"] = title_data["hurried"]

cleaned_data["district"] = post_data["district"]
cleaned_data["house_type"] = post_data["type"]

<h1><b>B. Replacing and Cleaning

Ở phần này, ta sẽ thay thế các data bị thiếu ở tập `main_data`, với thứ tự ưu tiên là `c_data` rồi `t_data`, vì các dữ liệu trong content sẽ được ghi chi tiết và đầy đủ hơn so với title.
Nếu như cả `c_data` và `t_data` đều trống, ta trả về `None` ở `main_data`

In [3]:
def replace_value(post, content, title):
    # Tạo màng gạt các giá trị null
    temp = content.copy()
    content_mask = content[content.isnull()].index.values
    
    title = title.loc[content_mask]    
    title_mask = title[title.isnull()].index.values

    title.loc[title_mask] = post.loc[title_mask]
    temp.loc[content_mask] = title.loc[content_mask]

    return temp

<h3><b>1. Price

Đối với giá tiền, trường hợp để trống dữ liệu sẽ không xảy ra, thay vào đó là "Thỏa thuận". Ngoài ra còn có các trường hợp nhập dư số không như 3899000000 tỷ hoặc 4500000 tỷ nhưng thực tế là 3.889 tỷ hoặc 4.5 tỷ

Giá trị của `price` nằm trong khoảng từ `1` đến nhỉnh hơn `1e+12`, trong đó từ giá trị từ `1` đến trước `200` sẽ được coi là chấp nhận được, các khoảng còn lại là bị lỗi nhập liệu. Cách sửa lỗi như sau:
- Từ `200` đến trước `1e6`: khoảng này rất đa dạng lỗi, nên sẽ không sửa và trả giá trị `None`
- Từ `1e6` đến trước `1e9`: khoảng nãy do người đăng nhập thêm 6 số 0, sửa bằng cách chia cho 1e6
- Từ `1e9` đến trước `1e12`: khoảng nãy do người đăng nhập thêm 9 số 0, sửa bằng cách chia cho 1e9
- `1e12` trở lên: khoảng nãy do người đăng nhập thêm 12 số 0, sửa bằng cách chia cho 1e12

In [4]:
def get_valid_price(price, thres = 350):
    if price != None:
        is_billions = re.search(r'(tỷ|tỉ)', str(price))        
        if not is_billions:
            has_number = re.search(r'\d+', str(price))
            if has_number:
                number = float(re.search(r'\d+', str(price)).group())
                if number > thres:
                    return price
                else:
                    return None
    return price

In [5]:
def clean_price(price, x=5):
    if type(price) == str:
        is_billions = re.search(r'(tỷ|tỉ)', price)
        
        if is_billions:
            normal_format = re.search(r'(\d+(\.\d+)?|\d+(\,\d+)?)', price)
            if normal_format:
                number = float(normal_format.group().replace(',', '.').replace('x', f'{x}'))
                # xử lý các trường hợp nhập dư số không
                if number > 1e+12:
                    number = number / 1e+12
                elif number < 1e+12 and number >= 1e+9:
                    number = number / 1e+9
                elif number < 1e+9 and number >= 1e+6:
                    number = number / 1e+6
                elif number >= 200 and number < 1e+6:
                    number = None
                return number
            else:
                number = float(price.replace(' ','').replace('tỷ','.').replace('tỉ','.'))
                return number

        elif not is_billions:
            is_millions = re.search(r'triệu', price)
            if is_millions:
                number = float(price.replace('triệu', '.').replace(' ', ''))
                return number / 1e+3

    return np.nan

In [6]:
post_price = post_data['price'].apply(get_valid_price)
content_price = content_data['price'].apply(get_valid_price)
title_price = title_data['price'].apply(get_valid_price)

cleaned_data["price"] = replace_value(post_price, content_price, title_price)
cleaned_data["price"] = cleaned_data["price"].apply(clean_price)

<h3><b>2. Square</h3>

In [7]:
post_square = post_data['square']
content_square = content_data['square']
title_square = title_data['square']

cleaned_data["square"] = replace_value(post_square, content_square, title_square)

<h3><b>3. Bedroom </h3>

In [8]:
post_bedroom = post_data['bedroom']
content_bedroom = content_data['bedrooms']
title_bedroom = title_data['bedrooms']

cleaned_data["bedroom"] = replace_value(post_bedroom, content_bedroom, title_bedroom)

<h3><b>4. Bathroom</h3>

In [9]:
post_bathroom = post_data['bathroom']
content_bathroom = content_data['bathrooms']
title_bathroom = title_data['bathrooms']

cleaned_data["bathroom"] = replace_value(post_bathroom, content_bathroom, title_bathroom)

<h3><b>5. House direction</h3>

In [10]:
post_house_direction = post_data['house_direction']
content_house_direction = content_data['house_direction']
title_house_direction = title_data['house_direction']

post_house_direction = post_house_direction.apply(lambda x: x.lower() if x is not np.nan else x)
cleaned_data["house_direction"] = replace_value(post_house_direction, content_house_direction, title_house_direction)

<h3><b>6. Balcony</h3>

In [11]:
post_balcony = post_data['balcony']
content_balcony = content_data['balcony']
title_balcony = title_data['balcony']

cleaned_data["has_balcony"] = replace_value(post_balcony, content_balcony, title_balcony)

C:\Users\dangm\AppData\Local\Temp\ipykernel_2436\3251895919.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  title.loc[title_mask] = post.loc[title_mask]
C:\Users\dangm\AppData\Local\Temp\ipykernel_2436\3251895919.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  temp.loc[content_mask] = title.loc[content_mask]


<h3><b>7. Furniture</h3>

In [12]:
post_furniture = post_data['furniture']
content_furniture = content_data['furniture']
title_furniture = title_data['furniture']

cleaned_data["has_furniture"] = replace_value(post_furniture, content_furniture, title_furniture)

C:\Users\dangm\AppData\Local\Temp\ipykernel_2436\3251895919.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  title.loc[title_mask] = post.loc[title_mask]
C:\Users\dangm\AppData\Local\Temp\ipykernel_2436\3251895919.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  temp.loc[content_mask] = title.loc[content_mask]


<h3><b>8. Legal information</h3>

In [13]:
post_legal_info = post_data['legal_info']
content_legal_info = content_data['legal_info']
title_legal_info = title_data['legal_info']

cleaned_data["has_legal_info"] = replace_value(post_legal_info, content_legal_info, title_legal_info)

C:\Users\dangm\AppData\Local\Temp\ipykernel_2436\3251895919.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  title.loc[title_mask] = post.loc[title_mask]
C:\Users\dangm\AppData\Local\Temp\ipykernel_2436\3251895919.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  temp.loc[content_mask] = title.loc[content_mask]


<h3><b>9. Hurried</h3>

In [14]:
is_hurried_mask = cleaned_data[(cleaned_data["c_hurried"] == True) | (cleaned_data["t_hurried"] == True)].index.values
cleaned_data["is_hurried"] = False
cleaned_data.loc[is_hurried_mask, "is_hurried"] = True

In [15]:
cleaned_data.head()

,post_code,post_date,content,title,c_rent,t_rent,c_hurried,t_hurried,district,house_type,price,square,bedroom,bathroom,house_direction,has_balcony,has_furniture,has_legal_info,is_hurried
0,285164,"[datetime.datetime(2023, 12, 10, 17, 11)]",Mô tả: \n+ Thông số cực đẹp 5 x 30m (150m2)...,Chưa tới 30tr/m2 - Hàng ngộp bank BAO ĐẦU TƯ ...,False,False,True,False,Nhà Bè,Nhà riêng,2.20,150.0,2.0,1.0,đông,False,False,True,True
1,285163,"[datetime.datetime(2023, 12, 10, 18, 40)]","Bán nhà HXH Âu Cơ Phường 9 Tân Bình, 51m2 3 Tầ...","Bán nhà HXH Âu Cơ Phường 9 Tân Bình, 51m2 3 Tầ...",False,False,False,False,Tân Bình,Nhà riêng,5.00,51.0,NaN,NaN,bắc,False,False,True,False
2,285162,"[datetime.datetime(2023, 12, 10, 18, 56)]","- Kết cấu: 1 trệt 1 lầu BTCT; 2 PN, 2 WC. Phòn...","SÁT MẶT TIỀN PHAN ĐĂNG LƯU, PHƯỜNG 7, PHÚ NHUẬ...",False,False,False,False,Phú Nhuận,Nhà,4.00,45.0,2.0,2.0,NaN,True,False,True,False
3,285161,"[datetime.datetime(2023, 12, 10, 20, 49)]","Mô tả:\n- Diện tích công nhận 41m, thực tế 66m...",CHỦ GẤP BÁN TRƯỚC TẾT LÊ HỒNG PHONG QUẬN 5 RA ...,True,False,False,True,Quận 5,Nhà riêng,7.35,41.0,NaN,NaN,NaN,False,False,True,True
4,285159,"[datetime.datetime(2023, 12, 7, 14, 13)]",- Diện tích 96m2 ngang 4.3m dài 22.5m .\n- sổ ...,"LŨY BÁN BÍCH,TÂN PHÚ-DIỆN TÍCH KHỦNG 96M2 ( 4....",False,False,False,False,Tân Phú,Nhà riêng,5.00,96.0,2.0,1.0,NaN,False,False,True,False


<h1><b>C. Loại bỏ giá trị nhiễu</h1>

<h3><b>1. Data lặp</h3>

Đôi khi các bài đăng sẽ bị đăng 2 lần, nên ta cần loại bỏ nó bằng cách xét nội dung bài đăng rồi tới tiêu đề

In [16]:
cleaned_data = cleaned_data.drop_duplicates(subset=["content", "title"], keep='first')

In [17]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9422 entries, 0 to 10048
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   post_code        9422 non-null   int64  
 1   post_date        9422 non-null   object 
 2   content          9422 non-null   object 
 3   title            9422 non-null   object 
 4   c_rent           9422 non-null   bool   
 5   t_rent           9422 non-null   bool   
 6   c_hurried        9422 non-null   bool   
 7   t_hurried        9422 non-null   bool   
 8   district         9422 non-null   object 
 9   house_type       9422 non-null   object 
 10  price            9139 non-null   float64
 11  square           9073 non-null   float64
 12  bedroom          6978 non-null   float64
 13  bathroom         6619 non-null   float64
 14  house_direction  3007 non-null   object 
 15  has_balcony      9422 non-null   object 
 16  has_furniture    9422 non-null   object 
 17  has_legal_info   9

<h3><b>2. Thuê nhà

Đôi khi có bài đăng nội dung chỉ thuê nhà, nên ta cần loại bỏ nó

In [18]:
rent_mask = cleaned_data[(cleaned_data["t_rent"] == True) & (cleaned_data["c_rent"] == True)].index.values
cleaned_data.drop(rent_mask, inplace=True)

<h3><b>3. Quảng cáo

Ngoài ra, cũng có các nội dung là về quảng cáo, tiếp thị mặt hàng khác, những thông tin này cũng sẽ loại bỏ

<h1><b>D. Xuất

In [19]:
cleaned_data.drop(['content', 'title', 'c_rent', 't_rent'], axis=1, inplace=True)

cleaned_data.to_csv('./dataset/csv_data/cleaned_data.csv', index=False, encoding="utf-8-sig")
print("Done!")

Done!
